# Preprocessing notebook for the PDO21 and PDO27 SPLiT-seq barcoding experiment datasets.

This notebook loads the raw annotated .h5ad files for each of the 2x PDO-21 and PDO-27 SPLiT-seq sublibraries and performs QC, filtering and cell-type clustering for further analysis and figure generation. The input data to the notebook and the final anndata output of the notebook can be found at this zenodo link: https://zenodo.org/records/8177571 

## Load packages and plot settings

In [ ]:
# Load packages
import scanpy as sc
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
from copy import copy
import numpy as np
import sklearn as sk
import scprep
import phate
import scrublet
import random
import anndata as ad

In [ ]:
# Setup the global plotting parameters
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.set_figure_params(dpi=100, color_map = "viridis", frameon=True, transparent=True,
                    dpi_save=800, facecolor="None", format="pdf", figsize=[4,4])

# Figure output directory
sc.settings.figdir = 'figures/combined_qc_figures'

# Set seed for reproducibility
np.random.seed(12)

## Load PDO21 and PDO27 datasets

In [ ]:
## Load 2x PDO-21 sublibraries
# Read fps
input_file_pdo21_s4 = '../../novaseq_run/split_tools_pdo21/exp013_p21_s4/filtered/h5ad_objects/exp013_p21_s4_sce_filtered.h5ad'  # h5ad file exported from preprocessing pipeline
input_file_pdo21_s5 = '../../novaseq_run/split_tools_pdo21/exp013_p21_s5/filtered/h5ad_objects/exp013_p21_s5_sce_filtered.h5ad'

# Import data into anndata object
split_p21_adata_s4 = sc.read_h5ad(input_file_pdo21_s4)
split_p21_adata_s5 = sc.read_h5ad(input_file_pdo21_s5)

# Swap names for gene_ids
p21_s4_var_df = split_p21_adata_s4.var
p21_s4_var_df['gene_names'] = p21_s4_var_df.index
p21_s4_var_df = p21_s4_var_df.set_index('gene_ids')
p21_s4_var_df.index.name = None
# Reassign
split_p21_adata_s4.var = p21_s4_var_df

# Swap names for gene_ids
p21_s5_var_df = split_p21_adata_s5.var
p21_s5_var_df['gene_names'] = p21_s5_var_df.index
p21_s5_var_df = p21_s5_var_df.set_index('gene_ids')
p21_s5_var_df.index.name = None
# Reassign
split_p21_adata_s5.var = p21_s5_var_df

# Merge into single anndata
# Observe the barcdoding dims after filtering
print("PDO_21_s4 data shape:", split_p21_adata_s4.shape)
print("PDO_21_s5 data shape:", split_p21_adata_s5.shape)

# Mege the two anndata objects 
adatas_pdo_21 = {
    "pdo_21_s4": split_p21_adata_s4,
    "pdo_21_s5": split_p21_adata_s5
}

# Merge
split_p21_adata = ad.concat(adatas_pdo_21, label=None, join="outer")

print("PDO_21 merged data shape:", split_p21_adata.shape)

In [ ]:
## Load 2x PDO-27 sublibraries
# Read fps
input_file_pdo27_s4 = '../../novaseq_run/split_tools_pdo27/exp013_p27_s4/filtered/h5ad_objects/exp013_p27_s4_sce_filtered.h5ad'  # h5ad file exported from preprocessing pipeline
input_file_pdo27_s5 = '../../novaseq_run/split_tools_pdo27/exp013_p27_s5/filtered/h5ad_objects/exp013_p27_s5_sce_filtered.h5ad'

# Import data into anndata object
split_p27_adata_s4 = sc.read_h5ad(input_file_pdo27_s4)
split_p27_adata_s5 = sc.read_h5ad(input_file_pdo27_s5)

# Swap names for gene_ids
p27_s4_var_df = split_p27_adata_s4.var
p27_s4_var_df['gene_names'] = p27_s4_var_df.index
p27_s4_var_df = p27_s4_var_df.set_index('gene_ids')
p27_s4_var_df.index.name = None
# Reassign
split_p27_adata_s4.var = p27_s4_var_df

# Swap names for gene_ids
p27_s5_var_df = split_p27_adata_s5.var
p27_s5_var_df['gene_names'] = p27_s5_var_df.index
p27_s5_var_df = p27_s5_var_df.set_index('gene_ids')
p27_s5_var_df.index.name = None
# Reassign
split_p27_adata_s5.var = p27_s5_var_df

# Merge into single anndata
# Observe the barcdoding dims after filtering
print("PDO_27_s4 data shape:", split_p27_adata_s4.shape)
print("PDO_27_s5 data shape:", split_p27_adata_s5.shape)

# Mege the two anndata objects 
adatas_pdo_27 = {
    "pdo_27_s4": split_p27_adata_s4,
    "pdo_27_s5": split_p27_adata_s5
}

# Merge
split_p27_adata = ad.concat(adatas_pdo_27, label=None, join="outer")

print("PDO_27 merged data shape:", split_p27_adata.shape)

## Filtering based on QC metrics per SPLiT-seq experiment

In [ ]:
# Extract MT gene names
# Load gene mapping tsv
gene_mapping_df = pd.read_csv("data/gene_names.txt", sep='\t')

# Pull mitochondiral gene ids based on gene name
mt_gene_df = gene_mapping_df[gene_mapping_df['gene_name'].str.startswith('MT-')]

# Convert to list
mt_gene_ids = mt_gene_df['gene_id'].tolist()

# Take a look
mt_gene_df

In [ ]:
# Create a boolean mask for mitochondrial genes
mito_mask_21 = [gene in mt_gene_ids for gene in split_p21_adata.var_names]
mito_mask_27 = [gene in mt_gene_ids for gene in split_p27_adata.var_names]

# Label mitochondiral genes and calculare QC metrics
split_p21_adata.var['mt'] = mito_mask_21  # annotate the group of mitochondrial genes as 'mt'
split_p27_adata.var['mt'] = mito_mask_27  # annotate the group of mitochondrial genes as 'mt'

# Label qc metics per cell
sc.pp.calculate_qc_metrics(split_p21_adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pp.calculate_qc_metrics(split_p27_adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
# create a dictionary to map cluster to annotation label
sampleid2condition = {
     'CAF_1': 'Fibroblast',
     'CAF_2': 'Fibroblast',
     'CAF_3': 'Fibroblast',
     'PDO_1': 'PDO',
     'PDO_2': 'PDO',
     'PDO_3': 'PDO',
     'PDO_CAF_1': 'Coculture',
     'PDO_CAF_2': 'Coculture',
     'PDO_CAF_3': 'Coculture',
}

# add a new `.obs` column called `condition` by mapping clusters to annotation using `map` function
split_p21_adata.obs['condition'] = split_p21_adata.obs['sample_id'].map(sampleid2condition).astype('category')
split_p27_adata.obs['condition'] = split_p27_adata.obs['sample_id'].map(sampleid2condition).astype('category')


In [ ]:
# Label the two PDO runs and culture conditions
pdo_21_vec = np.repeat(['pdo21'], '22689', axis=0)
pdo_27_vec = np.repeat(['pdo27'], '17163', axis=0)

split_p21_adata.obs['pdo'] = pdo_21_vec
split_p27_adata.obs['pdo'] = pdo_27_vec

# Create new condition with two cocatenated metadata values
cell_vec = split_p21_adata.obs['pdo'].astype('str')
condition_vec = split_p21_adata.obs['condition'].astype('str')
split_p21_adata.obs['cell_type_condition'] = cell_vec + "_" + condition_vec


cell_vec = split_p27_adata.obs['pdo'].astype('str')
condition_vec = split_p27_adata.obs['condition'].astype('str')
split_p27_adata.obs['cell_type_condition'] = cell_vec + "_" + condition_vec



In [ ]:
# PDO21 cell type filtering
# Filtering possible outliers based on IQR of total counts
# UMI IQR deviation by log10 value
umi_total_log10_21 = np.log10(split_p21_adata.obs.total_counts)
umi_med = umi_total_log10_21.median()

# Include new .obs for log10 counts
split_p21_adata.obs['log10_total_counts'] = umi_total_log10_21

# Computing IQR
Q1 = umi_total_log10_21.quantile(0.25)
Q3 = umi_total_log10_21.quantile(0.75)
IQR = Q3 - Q1
IQR

# UMI Cutoff for 2*IQR past the med of Log10
umi_cutoff_up = umi_med + (2 * IQR)

print("median log10(UMI): ", umi_med)
print("Upper log10(UMI): ", umi_cutoff_up)

# define outliers and filter the dataset
split_p21_adata.obs['outlier_mt'] = split_p21_adata.obs.pct_counts_mt > 20
split_p21_adata.obs['outlier_total'] = split_p21_adata.obs.log10_total_counts > umi_cutoff_up
split_p21_adata.obs['lowq_total'] = split_p21_adata.obs.total_counts < 1000

print('%u cells with high %% of mitochondrial genes' % (sum(split_p21_adata.obs['outlier_mt'])))
print('%u cells with large total counts' % (sum(split_p21_adata.obs['outlier_total'])))
print('%u cells with low number of total counts' % (sum(split_p21_adata.obs['lowq_total'])))


split_p21_adata = split_p21_adata[~split_p21_adata.obs['outlier_mt'], :]
split_p21_adata = split_p21_adata[~split_p21_adata.obs['outlier_total'], :]
split_p21_adata = split_p21_adata[~split_p21_adata.obs['lowq_total'], :]

In [ ]:
# Repeat filtering for PDO27
# Filtering possible outliers based on IQR of total counts
# UMI IQR deviation by log10 value
umi_total_log10_27 = np.log10(split_p27_adata.obs.total_counts)
umi_med = umi_total_log10_27.median()

# Include new .obs for log10 counts
split_p27_adata.obs['log10_total_counts'] = umi_total_log10_27

# Computing IQR
Q1 = umi_total_log10_27.quantile(0.25)
Q3 = umi_total_log10_27.quantile(0.75)
IQR = Q3 - Q1
IQR

# UMI Cutoff for 2*IQR past the med of Log10
umi_cutoff_up = umi_med + (2 * IQR)

print("median log10(UMI): ", umi_med)
print("Upper log10(UMI): ", umi_cutoff_up)

# Define outliers and filter the dataset
split_p27_adata.obs['outlier_mt'] = split_p27_adata.obs.pct_counts_mt > 20
split_p27_adata.obs['outlier_total'] = split_p27_adata.obs.log10_total_counts > umi_cutoff_up
split_p27_adata.obs['lowq_total'] = split_p27_adata.obs.total_counts < 1000

print('%u cells with high %% of mitochondrial genes' % (sum(split_p27_adata.obs['outlier_mt'])))
print('%u cells with large total counts' % (sum(split_p27_adata.obs['outlier_total'])))
print('%u cells with low number of total counts' % (sum(split_p27_adata.obs['lowq_total'])))


split_p27_adata = split_p27_adata[~split_p27_adata.obs['outlier_mt'], :]
split_p27_adata = split_p27_adata[~split_p27_adata.obs['outlier_total'], :]
split_p27_adata = split_p27_adata[~split_p27_adata.obs['lowq_total'], :]

In [ ]:
# Filter low complexity cells and lowly expressed genes
# PDO_21
split_p21_adata.var_names_make_unique()
print(split_p21_adata)
sc.pp.filter_cells(split_p21_adata, min_genes=400)
sc.pp.filter_genes(split_p21_adata, min_cells=25)

# PDO_27
split_p27_adata.var_names_make_unique()
print(split_p27_adata)
sc.pp.filter_cells(split_p27_adata, min_genes=400)
sc.pp.filter_genes(split_p27_adata, min_cells=25)

## Cell doublet cleanup with Scrublet

In [ ]:
# Estimated doublet rate at 3% based of prior data
sc.external.pp.scrublet(split_p21_adata, expected_doublet_rate=0.03, threshold=0.19)

In [ ]:
# Estimated doublet rate at 3% 
sc.external.pp.scrublet(split_p27_adata, expected_doublet_rate=0.03, threshold=0.18)

## Final Merging

In [ ]:
# Observe the barcoding dims after filtering
print("PDO_21 data shape:", split_p21_adata.shape)
print("PDO_27 data shape:", split_p27_adata.shape)

# Mege the two data objects 
adatas = {
    "pdo_21": split_p21_adata,
    "pdo_27": split_p27_adata
}

merged_adata = ad.concat(adatas, label="dataset", index_unique="_", join="outer")

In [ ]:
# Re-assign gene names from gene_ids
gene_names = gene_mapping_df.set_index('gene_id').loc[merged_adata.var_names, 'gene_name']
merged_adata.var['gene_names'] = gene_names

# stash gene_ids for later
merged_adata_var_df = merged_adata.var
merged_adata_var_df['gene_ids'] = merged_adata_var_df.index

# Reassign
merged_adata.var = merged_adata_var_df

# Check the var for correct names
merged_adata.var

In [ ]:
# Assign counts layer
merged_adata.layers['counts']=merged_adata.X.copy()

# Check merged data
merged_adata

## Generate phate to view doublets for cleanup

In [ ]:
# Count based normalization
sc.pp.normalize_total(merged_adata, target_sum=1e4, exclude_highly_expressed=True, max_fraction=0.05) 

# Log 
sc.pp.log1p(merged_adata)

# Find variable genes for downstream leiden clustering
sc.pp.highly_variable_genes( 
    merged_adata,
    n_top_genes=5000,
    flavor="seurat_v3",
    layer="counts",
    subset=False
)

In [ ]:
# Reassign gene names and make variables unique
merged_adata_var_df = merged_adata.var
merged_adata_var_df['gene_names'] = merged_adata_var_df['gene_names'].astype('str')
merged_adata_var_df = merged_adata_var_df.set_index('gene_names')
merged_adata_var_df.index.name = None

# Reassign
merged_adata.var = merged_adata_var_df

# Make names unique
merged_adata.var_names_make_unique()

In [ ]:
# Save log-norm data for downstream analysis
merged_adata.raw = merged_adata

In [ ]:
# Scale data for PCA and SNN clustering
sc.pp.scale(merged_adata, max_value=10) # Scale each gene to unit variance. Clip values exceeding standard deviation 10.
sc.tl.pca(merged_adata, n_comps=100, svd_solver='arpack')

In [ ]:
# Convert predicted doublet assignment to str
merged_adata.obs['predicted_doublet'] = merged_adata.obs['predicted_doublet'].astype(str).astype('category')

# Visualise PCA space
sc.pl.pca(merged_adata, color=['cell_type_condition', 'doublet_score', 'predicted_doublet'], components=['1,2'], ncols=1, save='_merged_pca_scrublet')

In [ ]:
# Run phate to inspect doublets
sc.external.tl.phate(merged_adata, t=7, random_state=12) 

In [ ]:
# View the RNA phate
sc.external.pl.phate(merged_adata, color=['cell_type_condition', 'doublet_score', 'predicted_doublet'], ncols=1,  
frameon=False, add_outline=True, title='', save="_all_pdo_doublets")

In [ ]:
# High level clustering to identify top-level cell types
sc.pp.neighbors(merged_adata, n_pcs=50, n_neighbors=100, random_state=12)
sc.tl.leiden(merged_adata, resolution = 0.1, random_state=12, key_added="leidenr0.1")
sc.external.pl.phate(merged_adata, color='leidenr0.1', use_raw=True, 
frameon=False, add_outline=True, title='', save="_merged_leiden0.1")

In [ ]:
# Get the leiden cluster labels
original_labels = merged_adata.obs['leidenr0.1'].values

new_cluster_names = {
    '0': 'PDO', '1': 'Fibroblast', '2':'PDO'
    }

# Create an array to store the new cluster labels
new_labels = np.empty_like(original_labels, dtype=object)

# Assign new cluster labels based on the mapping
for original_cluster, new_cluster in new_cluster_names.items():
    mask = original_labels == original_cluster
    new_labels[mask] = new_cluster

# Update the cluster labels in the anndata object
merged_adata.obs['cell_type'] = new_labels

# add a new .obs column
merged_adata.obs['condition'] = merged_adata.obs['sample_id'].map(sampleid2condition).astype('category')

# Merge the two conditions into a new .obs column
cell_type_vec = merged_adata.obs['cell_type'].astype('str')
condition_vec = merged_adata.obs['condition'].astype('str')

merged_adata.obs['cell_type_condition'] = cell_type_vec + "_" + condition_vec

In [ ]:
# Define outliers and filter 
merged_adata.obs['doublet_outliers'] = merged_adata.obs.predicted_doublet.isin(['True'])

print('%u cells with high probability of being doublets' % (sum(merged_adata.obs['doublet_outliers'])))

merged_adata = merged_adata[~merged_adata.obs['doublet_outliers'], :]

In [ ]:
# Remove CAF barcode collision doublets
# Create a boolean mask for the excluded values
exclude_values = ['PDO_Fibroblast']
mask = ~merged_adata.obs['cell_type_condition'].isin(exclude_values)

# Subset the anndata object using the boolean mask
PDO_Fibroblast_doublets  =  merged_adata[merged_adata.obs['cell_type_condition'].isin(exclude_values)]
merged_adata = merged_adata[mask]


In [ ]:
# View the Phate DR
sc.external.pl.phate(merged_adata, color=['cell_type_condition'], ncols=1,  
frameon=False, add_outline=True, title='', save="_all_pdo_doubblet_filtered")

In [ ]:
# Merge rep and condition at pdo and rep level, set as new .obs
pdo_vec = merged_adata.obs['pdo'].astype('str')
sample_id_vec = merged_adata.obs['sample_id'].astype('str')

merged_adata.obs['pdo_sample_id'] = pdo_vec + "_" + sample_id_vec

In [ ]:
# Annotate replicates in .obs
replicate_vec = [item.split("_")[-1] for item in sample_id_vec]
merged_adata.obs['pdo_replicate'] = pdo_vec + "_" + replicate_vec

# Rerun clustering for final global Phate embedding

In [ ]:
# Run phate
sc.external.tl.phate(merged_adata, t=7, random_state=12) 

In [ ]:
sc.tl.pca(merged_adata, n_comps=100, svd_solver='arpack')

# High level clustering to identfy cell types
sc.pp.neighbors(merged_adata, random_state=12, n_neighbors=100)
sc.tl.leiden(merged_adata, resolution = 0.1, random_state=12, key_added="leidenr0.1")


In [ ]:
# Get the original cluster labels
original_labels = merged_adata.obs['leidenr0.1'].values

new_cluster_names = {
    '0': 'PDO', '1': 'Fibroblast', '2':'PDO'
    }

# Create an array to store the new cluster labels
new_labels = np.empty_like(original_labels, dtype=object)

# Assign new cluster labels based on the mapping
for original_cluster, new_cluster in new_cluster_names.items():
    mask = original_labels == original_cluster
    new_labels[mask] = new_cluster

# Update the cluster labels in the AnnData object
merged_adata.obs['cell_type'] = new_labels

merged_adata.obs['condition'] = merged_adata.obs['sample_id'].map(sampleid2condition).astype('category')

# Merge the two conditions into new .obs
cell_type_vec = merged_adata.obs['cell_type'].astype('str')
condition_vec = merged_adata.obs['condition'].astype('str')

merged_adata.obs['cell_type_condition'] = cell_type_vec + "_" + condition_vec

In [ ]:
# Write object to file
merged_adata.write_h5ad(filename = 'merged_filtered_trellis_adata_v4.h5ad')